In [ ]:
from bs4 import BeautifulSoup as soup
from lxml import etree as ET
import pandas as pd
import re
#import requests

  * [Herangehensweise Datenbereinigung (falls genug Zeit)](#Herangehensweise-der-Datenbereinigung) 
  * [Fertige Funktion](#Die-fertige-Funktion)
  * [Erstellen der Visualisierung](#Erstellen-der-Visualisierung)

In [ ]:
# Load Dump: 

xml = open("dataset_tutorial_eco.xml", "r", encoding="utf-8")
data = soup(xml, "xml")                    

records = data.find_all("record", {'type':'Bibliographic'})
print("Gefundene Records:", len(records))

In [ ]:
# Show exmaple record: 

print(records[2].prettify())

## Herangehensweise der Datenbereinigung

In [ ]:
#check for dates: 

for record in records: 
    xml = ET.fromstring(str(record))
    date1 = xml.xpath(".//datafield[@tag = '264']/subfield[@code = 'c']")
    if date1:
        print(date1[0].text)
    else:
        print("N/A")

# modify and check more fields for date:

for record in records: 
    xml = ET.fromstring(str(record))
    date1 = xml.xpath(".//datafield[@tag = '264']/subfield[@code = 'c']")
    date2 = xml.xpath(".//datafield[@tag = '502']/subfield[@code = 'a']")
    if date1:
        print(date1[0].text)
    elif date2: 
        print(date2[0].text)
    else:
        print("N/A")

In [ ]:
# modify and only keep last 4 digits from dissertation statement:

for record in records: 
    xml = ET.fromstring(str(record))
    date1 = xml.xpath(".//datafield[@tag = '264']/subfield[@code = 'c']")
    date2 = xml.xpath(".//datafield[@tag = '502']/subfield[@code = 'a']")
    if date1:
        print(date1[0].text)
    elif date2: 
        date2 = date2[0].text
        date2 = date2[-4:]
        print(date2)
    else:
        print("N/A")

In [ ]:
# clean data: 

for record in records: 
    xml = ET.fromstring(str(record))
    date1 = xml.xpath(".//datafield[@tag = '264']/subfield[@code = 'c']")
    date2 = xml.xpath(".//datafield[@tag = '502']/subfield[@code = 'a']")
    if date1:
        date1 = date1[0].text
        date1 = date1.strip("[]")
        print(date1)
    elif date2: 
        date2 = date2[0].text
        date2 = date2[-4:]
        print(date2)
    else:
        print("N/A")

## Die fertige Funktion

In [ ]:
# Extract data: 

def get_data(record): 
        
    xml = ET.fromstring(str(record))
    
    date1 = xml.xpath(".//datafield[@tag = '264']/subfield[@code = 'c']")
    date2 = xml.xpath(".//datafield[@tag = '502']/subfield[@code = 'a']")
    if date1:
        date1 = date1[0].text
        date = date1.strip("[]c 'f idnn'")
        source = "264"
    elif date2:
        date2 = date2[0].text
        date = date2[-4:]
        source = "502"
    else:
        date = "N/A"
        
        
    idn = xml.xpath(".//controlfield[@tag = '001']")
    if idn:
        idn = idn[0].text
    
        
    results = {"date":date, "source":source, "idn":idn}
    
    return results
    
    #xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = '3']", namespaces=ns)

In [ ]:
data = [get_data(record) for record in records]
df = pd.DataFrame(data)
df

In [ ]:
# check for entries without a date: 

df.loc[df['date'] == "N/A"]

In [ ]:
df.query("date.str.len() > 4")

In [ ]:
df.to_excel("data/dates.xlsx")

## Erstellen der Visualisierung 

In [ ]:
import plotly.express as px

fig = px.bar(df, x='date')
fig.show()

In [ ]:
fig2 = px.histogram(df, x="date", barmode='group')
fig2.show()

In [ ]:
fig2 = px.histogram(df, x="date", barmode='group')
fig2.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig2.show()

In [ ]:
df.query('date.str.contains("1923")')